In [1]:
import re
from collections import defaultdict
import math
from tkinter import Tk, Label, Button, Entry, Text, Scrollbar, filedialog, END

In [2]:
class TextAnalyzer:
    def __init__(self):
        self.pair_freq = defaultdict(int)
        self.total_pairs = 0

    def preprocess_text(self, text):
        # Обробка тексту
        text = re.sub(r'[^а-яА-Я]', '', text.lower())
        return text

    def train(self, text):
        # Підрахунок пар букв
        text = self.preprocess_text(text)
        for i in range(len(text) - 1):
            pair = text[i:i+2]
            self.pair_freq[pair] += 1
            self.total_pairs += 1

    def calculate_probability(self, text):
        # Обчислюємо адекватність
        text = self.preprocess_text(text)
        pairs = [text[i:i+2] for i in range(len(text) - 1)]
        log_probability = 0.0
        for pair in pairs:
            pair_count = self.pair_freq.get(pair, 1)  # якщо пари нема в моделі, то використовуємо 1 для мінімального впливу
            probability = pair_count / self.total_pairs
            log_probability += math.log(probability)
        # Враховуємо кількість пар (тобто кількість символів - 1)
        normalized_score = log_probability / (len(pairs) or 1)
        # Перетворення в позитивну шкалу балів
        return math.exp(normalized_score) * 1000

In [3]:
class App:
    def __init__(self, root):
        self.analyzer = TextAnalyzer()
        self.root = root
        self.root.title("Text Analyzer")

        self.label = Label(root, text="Введіть рядок для аналізу:")
        self.label.pack()

        self.entry = Entry(root, width=50)
        self.entry.pack()

        self.result_label = Label(root, text="")
        self.result_label.pack()

        self.analyze_button = Button(root, text="Аналізувати", command=self.analyze_text)
        self.analyze_button.pack()

        self.load_button = Button(root, text="Завантажити текст для навчання", command=self.load_text)
        self.load_button.pack()

        self.train_label = Label(root, text="")
        self.train_label.pack()

        self.train_text = Text(root, wrap='word', height=10, width=50)
        self.train_text.pack()

        self.train_scroll = Scrollbar(root, command=self.train_text.yview)
        self.train_scroll.pack(side='right', fill='y')
        self.train_text['yscrollcommand'] = self.train_scroll.set

    def analyze_text(self):
        input_text = self.entry.get()
        score = self.analyzer.calculate_probability(input_text)
        if score < 2:
            self.result_label.config(text=f"Ймовірність: {score:.2f} балів. Текст неадекватний.")
        else:
            self.result_label.config(text=f"Ймовірність: {score:.2f} балів. Текст адекватний.")

    def load_text(self):
        file_path = filedialog.askopenfilename()
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            self.analyzer.train(text)
            self.train_label.config(text="Текст для навчання завантажено.")
            self.train_text.delete(1.0, END)
            self.train_text.insert(END, text)

if __name__ == "__main__":
    root = Tk()
    app = App(root)
    root.mainloop()
